In [21]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import models, transforms
from sklearn.model_selection import train_test_split
from PIL import Image
import os
from glob import glob

# === CONFIG ===
DATA_DIR = "../DataSet/Smaller Velocity Dataset/"
MODEL_SAVE_PATH = "../Models/velocity_regressor_split.pth"
BATCH_SIZE = 32
NUM_EPOCHS = 10
LEARNING_RATE = 1e-4

# === Dataset Definition ===
class VelocityDataset(Dataset):
    def __init__(self, images, velocities, transform=None):
        if images.ndim == 4 and images.shape[1] == 3:
            images = np.transpose(images, (0, 2, 3, 1))  # Convert (N, C, H, W) → (N, H, W, C)

        if images.max() <= 1.0:
            images = (images * 255).astype(np.uint8)

        self.images = images
        self.velocities = velocities
        self.transform = transform

    def __len__(self):
        return len(self.velocities)

    def __getitem__(self, idx):
        img = Image.fromarray(self.images[idx])
        velocity = self.velocities[idx]

        if self.transform:
            img = self.transform(img)

        return img, torch.tensor([velocity], dtype=torch.float32)

# === Transformations ===
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

# === Load and Combine All Data ===
image_files = sorted(glob(os.path.join(DATA_DIR, "*_images_Smaller.npy")))
velocity_files = sorted(glob(os.path.join(DATA_DIR, "*_velocity_Smaller.npy")))

all_images = []
all_velocities = []

for img_file, vel_file in zip(image_files, velocity_files):
    imgs = np.load(img_file)
    vels = np.load(vel_file)
    all_images.append(imgs)
    all_velocities.append(vels)

images = np.concatenate(all_images, axis=0)
velocities = np.concatenate(all_velocities, axis=0)

# === Split into Train / Validation / Test ===
X_temp, X_test, y_temp, y_test = train_test_split(
    images, velocities, test_size=0.1, random_state=42
)

X_train, X_val, y_train, y_val = train_test_split(
    X_temp, y_temp, test_size=0.1111, random_state=42
)
# Now: ~80% train, 10% val, 10% test

# === Create Datasets and Loaders ===
train_dataset = VelocityDataset(X_train, y_train, transform)
val_dataset   = VelocityDataset(X_val, y_val, transform)
test_dataset  = VelocityDataset(X_test, y_test, transform)

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader   = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)
test_loader  = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

# === Model Definition ===
class VelocityRegressor(nn.Module):
    def __init__(self):
        super().__init__()
        base = models.resnet18(pretrained=True)
        base.fc = nn.Linear(base.fc.in_features, 1)
        self.model = base

    def forward(self, x):
        return self.model(x)

# === Training Setup ===
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = VelocityRegressor().to(device)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)

# === Training Loop ===
for epoch in range(NUM_EPOCHS):
    model.train()
    train_loss = 0.0
    for images, targets in train_loader:
        images, targets = images.to(device), targets.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()
        train_loss += loss.item() * images.size(0)

    avg_train_loss = train_loss / len(train_loader.dataset)

    # === Validation ===
    model.eval()
    val_loss = 0.0
    with torch.no_grad():
        for images, targets in val_loader:
            images, targets = images.to(device), targets.to(device)
            outputs = model(images)
            loss = criterion(outputs, targets)
            val_loss += loss.item() * images.size(0)

    avg_val_loss = val_loss / len(val_loader.dataset)
    print(f"Epoch {epoch+1}/{NUM_EPOCHS} - Train Loss: {avg_train_loss:.4f} - Val Loss: {avg_val_loss:.4f}")

# === Save Model ===
torch.save(model.state_dict(), MODEL_SAVE_PATH)
print(f"✅ Model saved to {MODEL_SAVE_PATH}")


MemoryError: Unable to allocate 851. MiB for an array with shape (892861392,) and data type uint8